In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as Func

In [ ]:
spark = SparkSession.builder.appName("ModularCSVLoader").getOrCreate()

# Base URL for all the CSV files
base_url = "datasets/"

# List of file names to process
file_names = [
"allergies.csv",
"encounters.csv",
"medications.csv",
"patients.csv",
"procedures.csv"
]

# Dictionaries to store the resulting DataFrames for further processing/joining.
# pandas_dfs = {}
spark_dfs = {}

# Process each file and store the DataFrames into the dictionaries
for file_name in file_names:
    name_key = file_name.replace('.csv', '')
    file_url = f"{base_url}{file_name}"
    print(f"Processing file: {file_name}")
    spark_df = spark.read.csv(file_url, header=True, inferSchema=True)
    # pandas_dfs[name_key] = pd_df
    spark_dfs[name_key] = spark_df

In [ ]:
from pyspark.sql import functions as F

enc_df = spark_dfs['encounters'].filter(
    (spark_dfs['encounters'].REASONCODE == '55680006') &
    (spark_dfs['encounters'].START > F.lit("1999-07-15 00:00:00").cast("timestamp"))
)

enc_df.show(5)


In [ ]:
patient_df = spark_dfs['patients'].filter(
    (spark_dfs['patients'].BIRTHDATE.isNotNull())
)

patient_df.show(5)

In [ ]:
from pyspark.sql import functions as F

# Alias the DataFrames for clarity
pat = patient_df.alias("pat")
enc = enc_df.alias("enc")

# Join on patient Id. Use proper aliases when referring to columns.
joined_df = pat.join(enc, pat["Id"] == enc["PATIENT"], "inner")

# Select the columns of interest from each DataFrame and rename encounter columns to avoid ambiguity.
joined_df = joined_df.select(
    pat["Id"].alias("pat_Id"), enc['id'].alias("enc_Id"),
    pat["BIRTHDATE"], pat['DEATHDATE'],
    enc["PATIENT"].alias("enc_PATIENT_id"),
    enc["START"].alias("enc_START"),
    enc["STOP"].alias("enc_STOP")
)

# Calculate the patient's age at the time of the encounter.
# Assuming "BIRTHDATE" is in a format recognized by Spark for date conversion.
joined_df = joined_df.withColumn(
    "AGE_AT_VISIT",
    F.floor(F.datediff(F.col("enc_START").cast("date"), F.col("BIRTHDATE").cast("date")) / 365)
)

# Filter the records where age is between 18 and 35
cohort_df = joined_df.filter((F.col("AGE_AT_VISIT") >= 18) & (F.col("AGE_AT_VISIT") <= 35))

cohort_df.show(5)


In [ ]:

cohort_df = cohort_df.withColumn(
    "DEATH_AT_VISIT_IND",
    Func.when(
        # Check that DEATHDATE is not "NA" and falls between the START and STOP dates:
        (Func.col("DEATHDATE") != "NA") &
        (Func.to_timestamp(Func.col("DEATHDATE").cast("timestamp"), "yyyy-MM-dd HH:mm:ss").between(Func.col('enc_START').cast("timestamp"), Func.col('enc_STOP').cast("timestamp"))),
        1
    ).otherwise(0)
)

# For debugging or previewing results:
cohort_df.select("DEATHDATE", "DEATH_AT_VISIT_IND").show(5, truncate=False)


In [ ]:
# Filter records where DEATH_AT_VISIT_IND = 1
test_cohort_df = cohort_df.filter(Func.col("DEATH_AT_VISIT_IND") == 1)

# Show the filtered records
test_cohort_df.show(5, truncate=False)

In [ ]:
# COUNT_CURRENT_MEDS: Count of active medications at the start of the drug overdose encounter

from pyspark.sql import functions as F

# Alias the DataFrames for clarity
med = spark_dfs['medications'].alias("m")
cohort = cohort_df.alias("c")

# Join the DataFrames using their aliases for clarity in the join condition
med_df = med.join(cohort, F.col("c.pat_id") == F.col("m.PATIENT"), "inner") \
    .filter(
        F.col("m.START").cast("timestamp") >= F.col("c.enc_START").cast("timestamp")
    )

# Now use the actual column names for renaming; after join, med.PATIENT will appear as "PATIENT"
med_df = med_df.withColumnRenamed("PATIENT", "med_PATIENT") \
    .withColumnRenamed("START", "med_START") \
    .withColumnRenamed("STOP", "med_STOP") 

grouped_med = med_df.groupBy(
    "CODE", "ENCOUNTER", "med_PATIENT"
).agg(F.count("*").alias("med_cnt"))

grouped_med = grouped_med.groupBy("med_PATIENT", "ENCOUNTER").agg(
    F.sum("med_cnt").alias("COUNT_CURRENT_MEDS")
)

med_df = med_df.join(grouped_med,
    (med_df["med_PATIENT"] == grouped_med["med_PATIENT"]) &
    (med_df["ENCOUNTER"] == grouped_med["ENCOUNTER"]),
    "inner"
).select(
    med_df["*"],
    grouped_med["COUNT_CURRENT_MEDS"]
)

med_df.show(5, truncate=False)


In [ ]:
# Calcuale and add CURRENT_OPIOID_IND	
# if the patient had at least one active medication at the start of the overdose encounter that is on the Opioids List (provided below)

# GET CODES FOR  Opioids List:
# Hydromorphone 325Mg
# Fentanyl – 100 MCG
# Oxycodone-acetaminophen 100 Ml

from pyspark.sql import functions as F

patterns = [
    "(?i)^Hydromorphone 325", 
    "(?i)^Fentanyl", 
    "(?i)^Oxycodone-acetaminophen 100"
]

# Build a filter condition by OR-ing each pattern on the DESCRIPTION column
filter_condition = None
for pattern in patterns:
    cond = F.col("DESCRIPTION").rlike(pattern)
    filter_condition = cond if filter_condition is None else filter_condition | cond

# Filter med_df using the combined condition and return only the distinct CODE column.
# Then extract the CODE values as a Python list.
opioid_codes_list = [row["CODE"] for row in med_df.filter(filter_condition).select("CODE").distinct().collect()]

# Print the resulting list of opioid codes
print(opioid_codes_list)

# Add the CURRENT_OPIOID_IND column: 1 if med_df.CODE is in restricted_codes_list, else 0.
cohort_df = med_df.withColumn(
    "CURRENT_OPIOID_IND",
    F.when(F.col("CODE").isin(*opioid_codes_list), F.lit(1)).otherwise(F.lit(0))
)

# Show the results
cohort_df.show(5, truncate=False)

In [ ]:
# Filter records where DEATH_AT_VISIT_IND = 1
test_cohort_df = cohort_df.filter(Func.col("CURRENT_OPIOID_IND") == 1)

# Show the filtered records
test_cohort_df.show(5, truncate=False)

In [ ]:
cohort_df.printSchema()

In [ ]:
# Calculate and add READMISSION_90_DAY_IND	
# Indicator if the visit resulted in a subsequent readmission within 90 days

